In [2]:
import pandas as pd

# DataFrame 1: Business terms and their preferred forms
data1 = {
    'business_term': ['var', 'ROI', 'equity', 'profit'],
    'business_description': [
        'var in finance',
        'ROI for projects',
        'ownership in a company',
        'net income after expenses'
    ],
    'preferred_business_term': ['ValueAtRisk', 'ReturnOnInvestment', 'EquityShare', 'NetProfit'],
    'preferred_business_description': [
        'measure of the risk of loss for investments',
        'ratio between net profit and cost of investment',
        'value that represents part ownership in a corporation',
        'the amount of money that remains after all business expenses have been deducted from gross income'
    ]
}

df1 = pd.DataFrame(data1)

# DataFrame 2: Abbreviations and their full forms
data2 = {
    'abbreviation': ['var', 'ROI'],
    'full_form': ['value at risk', 'return on investment']
}

df2 = pd.DataFrame(data2)


In [3]:
abbreviation_dict = dict(zip(df2.abbreviation, df2.full_form))

def preprocess(text):
    # Handle abbreviations
    for abb, full in abbreviation_dict.items():
        text = text.replace(abb, full)

    # Tokenization, lower-casing, and filtering non-alphabetic tokens
    tokens = text.split()
    tokens = [token.lower() for token in tokens if token.isalpha()]

    return ' '.join(tokens)


In [1]:
!python -m spacy download en_core_web_md

2023-08-27 19:33:41.233965: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-27 19:33:42.042567: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 19.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [5]:
import spacy

# Load the medium English model from Spacy
nlp = spacy.load('en_core_web_md')

def get_embedding(text):
    return nlp(text).vector


In [6]:
from sklearn.metrics.pairwise import cosine_similarity

def get_most_similar(user_input, df):
    user_embedding = get_embedding(preprocess(user_input))
    similarities = []

    for index, row in df.iterrows():
        term_embedding = get_embedding(preprocess(row['preferred_business_description']))
        sim = cosine_similarity([user_embedding], [term_embedding])[0][0]
        similarities.append((row['preferred_business_term'], sim))

    # Sort by similarity and return top 4
    sorted_terms = sorted(similarities, key=lambda x: x[1], reverse=True)
    return [term[0] for term in sorted_terms[:4]]


In [7]:
user_input = "What is the return for my investment project?"
top_terms = get_most_similar(user_input, df1)
top_terms

['NetProfit', 'EquityShare', 'ValueAtRisk', 'ReturnOnInvestment']

In [8]:
def get_most_similar_v2(business_term, business_description, df):
    # Combine business term and description and get its embedding
    user_input = business_term + " " + business_description
    user_embedding = get_embedding(preprocess(user_input))
    similarities = []

    for index, row in df.iterrows():
        # Combine preferred business term and description from the dataframe and get its embedding
        combined_description = row['business_term'] + " " + row['business_description']
        term_embedding = get_embedding(preprocess(combined_description))
        sim = cosine_similarity([user_embedding], [term_embedding])[0][0]
        similarities.append((row['preferred_business_term'], sim))

    # Sort by similarity and return top 4
    sorted_terms = sorted(similarities, key=lambda x: x[1], reverse=True)
    return [term[0] for term in sorted_terms[:4]]

# Testing with a user input for both business term and business description
user_business_term = "ROI"
user_business_description = "ratio of net gain in the project"
top_terms_v2 = get_most_similar_v2(user_business_term, user_business_description, df1)
top_terms_v2


['ReturnOnInvestment', 'EquityShare', 'ValueAtRisk', 'NetProfit']

In [9]:
data1

{'business_term': ['var', 'ROI', 'equity', 'profit'],
 'business_description': ['var in finance',
  'ROI for projects',
  'ownership in a company',
  'net income after expenses'],
 'preferred_business_term': ['ValueAtRisk',
  'ReturnOnInvestment',
  'EquityShare',
  'NetProfit'],
 'preferred_business_description': ['measure of the risk of loss for investments',
  'ratio between net profit and cost of investment',
  'value that represents part ownership in a corporation',
  'the amount of money that remains after all business expenses have been deducted from gross income']}

In [10]:


# DataFrame 1: Business terms
data_terms = {
    'business_term': ['var', 'ROI', 'equity', 'profit'],
    'business_description': [
        'var in finance',
        'ROI for projects',
        'ownership in a company',
        'net income after expenses'
    ]
}
df_terms = pd.DataFrame(data_terms)

# DataFrame 2: Preferred business terms
data_preferred_terms = {
    'preferred_business_term': ['ValueAtRisk', 'ReturnOnInvestment', 'EquityShare', 'NetProfit'],
    'preferred_business_description': [
        'measure of the risk of loss for investments',
        'ratio between net profit and cost of investment',
        'value that represents part ownership in a corporation',
        'the amount of money that remains after all business expenses have been deducted from gross income'
    ]
}
df_preferred_terms = pd.DataFrame(data_preferred_terms)

# DataFrame 3: Abbreviations
data_abbreviations = {
    'abbreviation': ['var', 'ROI'],
    'full_form': ['value at risk', 'return on investment']
}
df_abbreviations = pd.DataFrame(data_abbreviations)

# Abbreviation dictionary from df_abbreviations
abbreviation_dict = dict(zip(df_abbreviations.abbreviation, df_abbreviations.full_form))

def preprocess(text):
    # Handle abbreviations
    for abb, full in abbreviation_dict.items():
        text = text.replace(abb, full)
    tokens = text.split()
    tokens = [token.lower() for token in tokens if token.isalpha()]
    return ' '.join(tokens)

# Pre-compute embeddings for df_preferred_terms
precomputed_embeddings = [get_embedding(preprocess(desc)) for desc in df_preferred_terms['preferred_business_description']]

def get_most_similar_optimized_v2(business_term, business_description):
    # Combine business term and description and get its embedding
    user_input = business_term + " " + business_description
    user_embedding = get_embedding(preprocess(user_input))

    # Compute the cosine similarity in batch
    similarities = cosine_similarity([user_embedding], precomputed_embeddings)[0]

    # Pair preferred business terms with their similarities and sort them
    sorted_terms = sorted(list(zip(df_preferred_terms['preferred_business_term'], similarities)), key=lambda x: x[1], reverse=True)

    return [term[0] for term in sorted_terms[:4]]

# Testing with the new approach
user_business_term = "ROI"
user_business_description = "ratio of net gain in the project"
top_terms_optimized_v2 = get_most_similar_optimized_v2(user_business_term, user_business_description)
top_terms_optimized_v2


['ValueAtRisk', 'ReturnOnInvestment', 'NetProfit', 'EquityShare']

In [3]:
# !pip install transformers
# !pip install torch
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 5.5 MB/s eta 0:00:00


In [5]:
import numpy as np
import pandas as pd
import faiss
from transformers import BertTokenizer, BertModel
import torch

# Define the three dataframes

# Mock data for df_terms
data_terms = {
    'business_term': ['var', 'ROI', 'equity', 'profit'],
    'business_description': [
        'var in finance',
        'ROI for projects',
        'ownership in a company',
        'net income after expenses'
    ]
}
df_terms = pd.DataFrame(data_terms)

# Mock data for df_preferred_terms
data_preferred_terms = {
    'preferred_business_term': ['ValueAtRisk', 'ReturnOnInvestment', 'EquityShare', 'NetProfit', 'aojsdsd'],
    'preferred_business_description': [
        'measure of the risk of loss for investments',
        'ratio between net profit and cost of investment',
        'value that represents part ownership in a corporation',
        'the amount of money that remains after all business expenses have been deducted from gross income',
        'oeijsdfds'
    ]
}
df_preferred_terms = pd.DataFrame(data_preferred_terms)

# Mock data for df_abbreviations
data_abbreviations = {
    'abbreviation': ['var', 'ROI'],
    'full_form': ['value at risk', 'return on investment']
}
df_abbreviations = pd.DataFrame(data_abbreviations)

# Preprocessing: Abbreviation expansion
abbreviation_dict = dict(zip(df_abbreviations.abbreviation, df_abbreviations.full_form))

def preprocess(text):
    for abb, full in abbreviation_dict.items():
        text = text.replace(abb, full)
    return text

# Initialize BERT model and tokenizer
model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Function to get BERT embeddings
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs['pooler_output'].numpy()

# Precompute embeddings for df_preferred_terms
embeddings = []
for _, row in df_preferred_terms.iterrows():
    term_embedding = get_bert_embedding(preprocess(row['preferred_business_term']))
    desc_embedding = get_bert_embedding(preprocess(row['preferred_business_description']))
    combined_embedding = (term_embedding + desc_embedding) / 2.0
    embeddings.append(combined_embedding.squeeze())

# Convert embeddings list to a matrix
embedding_matrix = np.vstack(embeddings)

# Build a Faiss index
index = faiss.IndexFlatL2(embedding_matrix.shape[1])
index.add(embedding_matrix.astype('float32'))

# Enhanced search function
def enhanced_search(business_term, business_description, top_k=4):
    term_embedding = get_bert_embedding(preprocess(business_term))
    desc_embedding = get_bert_embedding(preprocess(business_description))
    query_embedding = ((term_embedding + desc_embedding) / 2.0).astype('float32')
    D, I = index.search(query_embedding, top_k)
    return [df_preferred_terms.iloc[i]['preferred_business_term'] for i in I[0]]

# Testing the enhanced search
user_business_term = "ROI"
user_business_description = "ratio of net gain in the project"
top_preferred_terms = enhanced_search(user_business_term, user_business_description)
top_preferred_terms


['NetProfit', 'EquityShare', 'ValueAtRisk', 'ReturnOnInvestment']

In [2]:
import spacy
import pandas as pd

# Full dataframes
data1 = {
    'Business Term': [
        "Primary Key", "Counter Party Identifier", "Foreign Key", "Facility", "Counterparty",
        "Risk Type", "Residential Address", "Basel Committee", "Financial Instrument",
        "Customer Identifier", "Trade Transaction", "Credit Exposure", "Product Category",
        "Regulatory Authority", "Investment Portfolio", "Contractual Agreement",
        "Tax Identification Number", "Market Participant", "Credit Rating", "Legal Entity",
        "Portfolio Management", "Country Risk", "Trade Agreement", "Credit Analyst",
        "Regulatory Compliance", "Market Volatility", "Income Statement", "Equity Investment",
        "Risk Mitigation", "Invoice Processing", "Economic Indicator", "Vendor Relationship",
        "Securities Exchange", "Product Catalog", "Asset Valuation", "Loan Agreement",
        "Central Bank", "Market Share", "Taxation Policy", "Risk Assessment",
        "Partnership Agreement", "Monetary Policy", "Business Strategy", "Real Estate Investment",
        "Stock Market", "Consumer Behavior", "Derivative Contract"
    ],
    'Business Term Description': [
        "A unique identifier for an object", "An identifier uniquely identifying a country",
        "A reference that differentiates an object", "A financial instrument for credit and risk control",
        "Individual or organization in trade or transaction", "A classification of risk by its nature",
        "Address associated with an individual residing", "Organization developing banking standards",
        "A contract for a financial obligation", "An identifier that uniquely identifies a customer",
        "An agreement for exchanging goods or services", "The extent of credit risk",
        "A classification of products based on attributes", "Authority responsible for regulations",
        "Collection of assets for investment", "A legally binding arrangement",
        "Identifier for tax purposes", "Entity engaged in market transactions",
        "Evaluation of creditworthiness", "Organization with legal rights and responsibilities",
        "Management of financial assets", "Risk associated with a specific country",
        "Contract for exchanging goods or services", "Expert assessing creditworthiness",
        "Adherence to regulatory requirements", "Measure of price fluctuations in the market",
        "Financial report detailing revenues and expenses", "Ownership stake in a company",
        "Strategies to reduce potential losses", "Handling of invoices for payment",
        "Measurement reflecting economic trends", "Business relationship with a supplier",
        "Market where securities are bought and sold", "Collection of available products",
        "Determination of the value of assets", "Contract for lending and borrowing",
        "National financial institution with regulatory responsibilities", "Proportion of the market held by a company",
        "Regulations and laws regarding taxation", "Evaluation of potential risks",
        "Contract between partners for collaboration", "Strategies by a central bank to control the money supply",
        "Plan of action to achieve business goals", "Investment in property and land",
        "Venue for buying and selling company shares", "Study of how individuals make purchasing decisions",
        "Financial contract derived from an underlying asset"
    ]
}

data2 = {
    'Preferred Business Term': [
        "Object Identifier", "Country Identifier", "Differentiating Reference", "Credit and Risk Control",
        "Trade Partner", "Nature of Risk", "Individual Address", "Banking Standards Organization",
        "Contract for Financial Obligation", "Customer ID", "Exchange Agreement", "Risk Extent",
        "Attribute-Based Classification", "Regulations Oversight", "Asset Investment Collection",
        "Legally Binding Arrangement", "Tax ID", "Market Transactions Participant",
        "Creditworthiness Evaluation", "Entity with Legal Rights", "Financial Assets Management",
        "Specific Country Risk", "Goods Exchange Contract", "Creditworthiness Assessor",
        "Requirements Adherence", "Price Fluctuation Measurement", "Revenues and Expenses Report",
        "Company Ownership Stake", "Potential Loss Reduction", "Payment Invoice Handling",
        "Economic Trend Measurement", "Supplier Business Relationship", "Securities Trading Market",
        "Available Products Collection", "Asset Value Determination", "Lending and Borrowing Contract",
        "Financial Regulation Authority", "Company Market Proportion", "Tax Regulations and Laws",
        "Potential Risk Evaluation", "Collaborative Partnership", "Money Supply Control Strategy",
        "Goal Achievement Plan", "Property Investment", "Company Shares Trading Venue",
        "Purchasing Decision Study", "Underlying Asset-Derived Contract"
    ],
    'Preferred Business Term Description': [
        "A unique identifier for an object", "An identifier uniquely identifying a country",
        "A reference that differentiates an object", "A financial instrument for credit and risk control",
        "Individual or organization in trade or transaction", "A classification of risk by its nature",
        "Address associated with an individual residing", "Organization developing banking standards",
        "A contract for a financial obligation", "An identifier that uniquely identifies a customer",
        "An agreement for exchanging goods or services", "The extent of credit risk",
        "A classification of products based on attributes", "Authority responsible for regulations",
        "Collection of assets for investment", "A legally binding arrangement",
        "Identifier for tax purposes", "Entity engaged in market transactions",
        "Evaluation of creditworthiness", "Organization with legal rights and responsibilities",
        "Management of financial assets", "Risk associated with a specific country",
        "Contract for exchanging goods or services", "Expert assessing creditworthiness",
        "Adherence to regulatory requirements", "Measure of price fluctuations in the market",
        "Financial report detailing revenues and expenses", "Ownership stake in a company",
        "Strategies to reduce potential losses", "Handling of invoices for payment",
        "Measurement reflecting economic trends", "Business relationship with a supplier",
        "Market where securities are bought and sold", "Collection of available products",
        "Determination of the value of assets", "Contract for lending and borrowing",
        "National financial institution with regulatory responsibilities", "Proportion of the market held by a company",
        "Regulations and laws regarding taxation", "Evaluation of potential risks",
        "Contract between partners for collaboration", "Strategies by a central bank to control the money supply",
        "Plan of action to achieve business goals", "Investment in property and land",
        "Venue for buying and selling company shares", "Study of how individuals make purchasing decisions",
        "Financial contract derived from an underlying asset"
    ]
}

df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

# [Rest of the code remains the same]


In [5]:
# Load the spaCy model
nlp = spacy.load("en_core_web_md")

# Sample abbreviation dictionary (expand as needed)
abbr_dict = {
    "var": "value at risk"
    # Add other abbreviations as needed
}

def expand_abbreviations(text):
    for abbr, expansion in abbr_dict.items():
        text = text.replace(abbr, expansion)
    return text

# Preprocess the text from df2
df2['Processed_Description'] = df2['Preferred Business Term Description'].apply(lambda x: nlp(expand_abbreviations(x)))

def get_top_matches(user_input_term, user_input_description):
    # Process the user input
    user_input = nlp(expand_abbreviations(user_input_term + " " + user_input_description))

    # Calculate similarity scores
    df2['Similarity'] = df2['Processed_Description'].apply(lambda x: x.similarity(user_input))

    # Sort by similarity and get top 4 matches
    top_matches = df2.sort_values(by='Similarity', ascending=False).head(4)

    return top_matches['Preferred Business Term'].tolist()

# Example usage
user_input_term = "Credit score"
user_input_description = "Checking if a candidate is credit worthy"
results = get_top_matches(user_input_term, user_input_description)
print(results)

['Credit and Risk Control', 'Supplier Business Relationship', 'Contract for Financial Obligation', 'Specific Country Risk']


In [6]:
import spacy
import pandas as pd
import numpy as np

# Your dataframes go here (df1, df2)

# Load the spaCy model
nlp = spacy.load("en_core_web_md")

# Sample abbreviation dictionary (expand as needed)
abbr_dict = {
    "var": "value at risk"
    # Add other abbreviations as needed
}

def expand_abbreviations(text):
    for abbr, expansion in abbr_dict.items():
        text = text.replace(abbr, expansion)
    return text

# Preprocess the text from df2
df2['Processed_Description'] = df2['Preferred Business Term Description'].apply(lambda x: nlp(expand_abbreviations(x)))

# Cosine similarity function
def cosine_similarity(A, B):
    dot_product = np.dot(A, B)
    norm_a = np.linalg.norm(A)
    norm_b = np.linalg.norm(B)
    return dot_product / (norm_a * norm_b)

def get_top_matches(user_input_term, user_input_description):
    # Process the user input
    user_input = nlp(expand_abbreviations(user_input_term + " " + user_input_description))

    # Calculate cosine similarity scores
    df2['Similarity'] = df2['Processed_Description'].apply(lambda x: cosine_similarity(x.vector, user_input.vector))

    # Sort by similarity and get top 4 matches
    top_matches = df2.sort_values(by='Similarity', ascending=False).head(4)

    return top_matches['Preferred Business Term'].tolist()

# Example usage
user_input_term = "Credit score"
user_input_description = "Checking if a candidate is credit worthy"
results = get_top_matches(user_input_term, user_input_description)
print(results)


['Credit and Risk Control', 'Supplier Business Relationship', 'Contract for Financial Obligation', 'Specific Country Risk']


In [9]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertModel
import torch

# Your dataframes go here (df1, df2)

# Load the BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to get BERT embeddings
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    # Use mean of the last hidden state as the sentence representation
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.numpy()

# Cosine similarity function
# Cosine similarity function
def cosine_similarity(A, B):
    A = A.flatten()
    B = B.flatten()
    dot_product = np.dot(A, B)
    norm_a = np.linalg.norm(A)
    norm_b = np.linalg.norm(B)
    return dot_product / (norm_a * norm_b)


def get_top_matches(user_input_term, user_input_description):
    # Get BERT embedding for the user input
    user_input_embedding = get_bert_embedding(user_input_term + " " + user_input_description)

    # Calculate cosine similarity scores
    df2['Similarity'] = df2['Preferred Business Term Description'].apply(lambda x: cosine_similarity(get_bert_embedding(x), user_input_embedding))

    # Sort by similarity and get top 4 matches
    top_matches = df2.sort_values(by='Similarity', ascending=False).head(4)

    return top_matches['Preferred Business Term'].tolist()

# Example usage
user_input_term = "Credit score"
user_input_description = "Checking if a candidate is credit worthy"
results = get_top_matches(user_input_term, user_input_description)
print(results)


['Creditworthiness Evaluation', 'Creditworthiness Assessor', 'Credit and Risk Control', 'Risk Extent']
